<a href="https://colab.research.google.com/github/pablomiralles22/class-CV-computer-vision/blob/main/YOLO_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch torchvision matplotlib pillow

In [ ]:
import torch
import requests
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from transformers import YolosImageProcessor, YolosForObjectDetection
from PIL import Image

In [ ]:
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

model = YolosForObjectDetection.from_pretrained('hustvl/yolos-tiny')
image_processor = YolosImageProcessor.from_pretrained("hustvl/yolos-tiny")

inputs = image_processor(images=image, return_tensors="pt")
outputs = model(**inputs)

# model predicts bounding boxes and corresponding COCO classes
logits = outputs.logits
bboxes = outputs.pred_boxes

# print results
target_sizes = torch.tensor([image.size[::-1]])
results = image_processor.post_process_object_detection(outputs, threshold=0.9, target_sizes=target_sizes)[0]
for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    print(
        f"Detected {model.config.id2label[label.item()]} with confidence "
        f"{round(score.item(), 3)} at location {box}"
    )

In [ ]:
# Plot the image
plt.figure(figsize=(12, 8))
plt.imshow(image)
ax = plt.gca()

# Add each bounding box and label
for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    xmin, ymin, xmax, ymax = box.tolist()
    width, height = xmax - xmin, ymax - ymin
    rect = patches.Rectangle(
        (xmin, ymin), width, height,
        linewidth=2, edgecolor="red", facecolor="none"
    )
    ax.add_patch(rect)
    label_text = f"{model.config.id2label[label.item()]}: {score.item():.2f}"
    ax.text(xmin, ymin - 5, label_text, color="white",
            fontsize=12, bbox=dict(facecolor="red", alpha=0.5))

plt.axis("off")
plt.show()